In [ ]:
import os
import torch
import pandas as pd
from modules.model import Dataloader
from modules.utils import load_dataset
from modules.experiment import run_experiment
from modules.argument import TrainArguments

In [ ]:
def train(model = None, **train_args):
    args = TrainArguments(**train_args)
    # Setup torch
    device = torch.device(args.device if torch.cuda.is_available() else 'cpu')
    # Load dataset
    g, features, ano_label, _, _ = load_dataset(args.dataset)
    features = torch.FloatTensor(features)
    if args.batch_size == -1:
        features = features.to(device)
    g = g.to(device)
    dataloader = Dataloader(g, features, args.k, dataset_name=args.dataset)
    if not os.path.isdir("./ckpt"):
        os.makedirs("./ckpt")

    # Run the experiment
    seed = args.seed
    model, stats = run_experiment(args, seed, device, dataloader, ano_label, model)
    print("AUC: %.4f" % stats["AUC"])
    # print("Time (Train): %.4fs" % stats["time_train"])
    # print("Mem (Train): %.4f MB" % (stats["mem_train"] / 1024 / 1024))
    # print("Time (Test): %.4fs" % stats["time_test"])
    # print("Mem (Test): %.4f MB" % (stats["mem_test"] / 1024 / 1024))
    return stats

In [ ]:
data = pd.read_csv("./record.csv", index_col=0)
data.columns = data.columns.astype(float)

for alpha in range(1, 10):
    alpha = alpha * 0.1
    for gamma in range(1, 10):
        gamma = gamma * 0.1
        print(f"alpha:{alpha:.1f}, gamma:{gamma:.1f}")
        stats = train(dataset="cora", lr=0.01, alpha=alpha, gamma=gamma, num_epoch=100, batch_size=1024)
        data.loc[round(alpha, 1), round(gamma, 1)] = round(stats["AUC"], 4)
        del stats
        data.to_csv("./record.csv")